In [116]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [117]:
import statsmodels.api as sm
import statsmodels.formula.api as sf

In [118]:
# 서열척도
col1=[('(가)건강 만족도', '(다)주거 환경 만족도'), ('(나)가족의 수입 만족도', '교육수준'),
       ('(다)주거 환경 만족도', '교육수준'), ('(라)가족관계 만족도', '교육수준'),
       ('(마)직업 만족도', '교육수준'), ('(바)사회적 친분관계 만족도', '교육수준'),
       ('(사)여가생활 만족도', '교육수준')]
# 카이제곱
col2=[['(다)주거 환경 만족도', '교육수준'],['(다)주거 환경 만족도', '종교'],
       ['(다)주거 환경 만족도', '만성질환'],['(다)주거 환경 만족도', '근로유형']]
# 범주~연속형(anova)
col3=[['(나)가족의 수입 만족도', '방수'],['(다)주거 환경 만족도', '연령'],
       ['(마)직업 만족도', '방수'],['(바)사회적 친분관계 만족도', '방수'],
       ['(사)여가생활 만족도', '방수']]
# 연속형~연속형(regression)
col4=[['방수','연령'],['연령','주거면적'],['총생활비','방수'],['주거면적','총생활비'],['총생활비','주거면적']]
c1=list(sum(list(map(list,col1)),[]))
c1.extend(list(set(sum(list(map(list,col2)),[]))))
c2=list(sum(list(map(list,col3)),[]))
c3=list(sum(list(map(list,col4)),[]))

In [119]:
D=pd.read_excel("../02_데이터검정/카테고리_소분류완료.xlsx",usecols=list(set(c1+c2+c3))+["삶의 사다리 점수"])
data=pd.DataFrame(D)
# 명목형의 데이터 형태 바꾸기
for col in ["근로유형","종교","만성질환"]:
    data[col]=data[col].astype("category")

## train,test 데이터 분리

In [120]:
train_x,test_x,train_y,test_y=train_test_split(data[data.columns.difference(["삶의 사다리 점수"])],data["삶의 사다리 점수"],train_size=0.7,stratify=data["삶의 사다리 점수"],random_state=10)

In [121]:
# logist regression(1)
test=pd.DataFrame(train_y.apply(lambda x:1 if x==1 else 0))

In [122]:
fit=sm.Logit(endog=train_y.apply(lambda x:1 if x==1 else 0),exog=pd.get_dummies(sm.add_constant(train_x),drop_first=True,dtype='int')).fit()

         Current function value: 0.029597
         Iterations: 35


c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [123]:
result=pd.DataFrame(fit.fittedvalues)

In [124]:
fit.fittedvalues.apply(np.exp)

3206    5.293854e-05
934     2.800431e-14
2919    1.058362e-04
2937    1.466708e-05
1501    1.192793e-01
            ...     
2627    5.155985e-02
1151    6.699144e-04
137     7.186429e-05
584     7.872005e-03
868     9.332531e-03
Length: 2333, dtype: float64

In [125]:
설명2=pd.DataFrame(fit.predict(pd.get_dummies(sm.add_constant(train_x),drop_first=True,dtype='int')))

In [126]:
설명2[설명2[0]>=0.5]

,0
2460,0.531080
142,0.732285
319,0.599736


In [127]:
설명=pd.merge(result,test,right_index=True,left_index=True)
설명[0].apply(np.exp)

3206    5.293854e-05
934     2.800431e-14
2919    1.058362e-04
2937    1.466708e-05
1501    1.192793e-01
            ...     
2627    5.155985e-02
1151    6.699144e-04
137     7.186429e-05
584     7.872005e-03
868     9.332531e-03
Name: 0, Length: 2333, dtype: float64

In [128]:
설명[0]=설명[0].apply(np.exp).apply(lambda x:x/(1+x))

In [129]:
설명[설명[0]>=0.5]

,0,삶의 사다리 점수
2460,0.531080,1
142,0.732285,1
319,0.599736,0


In [130]:
acc=(fit.pred_table()[0,0]+fit.pred_table()[1,1])/fit.pred_table().sum()

In [131]:
aic=fit.aic

In [132]:
result=[]
for num in range(1,11):
    temp={}
    train_x=sm.add_constant(train_x)
    #카테고리형 x는 get_dummies
    fit=sm.Logit(endog=train_y.apply(lambda x:1 if x==num else 0),exog=pd.get_dummies(sm.add_constant(train_x),drop_first=True,dtype='int')).fit()
    temp["accuracy"]=(fit.pred_table()[0,0]+fit.pred_table()[1,1])/fit.pred_table().sum()
    temp["aic"]=fit.aic
    temp["model"]=fit
    result.append(temp)

         Current function value: 0.029597
         Iterations: 35
         Current function value: 0.066573
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.168493
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.263871
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.470567
         Iterations 7


c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.502396
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.459089
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.378541
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.178855
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.116335
         Iterations 9


In [133]:
def logit(y,x):
    result=[]
    # 1~10 사이의 값을
    for num in y.unique():
        temp={}
        x=sm.add_constant(x)
        #카테고리형 x는 get_dummies
        fit=sm.Logit(endog=y.apply(lambda x:1 if x==num else 0),exog=pd.get_dummies(sm.add_constant(x),drop_first=True,dtype='int')).fit()
        temp["accuracy"]=(fit.pred_table()[0,0]+fit.pred_table()[1,1])/fit.pred_table().sum()
        temp["aic"]=fit.aic
        temp["model"]=fit
        result.append(temp)
    return result

In [134]:
models=[x["model"] for x in logit(train_y,train_x[['(가)건강 만족도', '(다)주거 환경 만족도']])]

Optimization terminated successfully.
         Current function value: 0.401511
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.470806
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.040637
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.507291
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.275938
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.485517
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.192580
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.189577
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.131899
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.075925
 

In [141]:
test=pd.DataFrame({i:models[i].fittedvalues for i in range(10) }).apply(np.exp)

In [143]:
test["max"]=test.apply(max,axis=1)

In [144]:
test

,0,1,2,3,4,5,6,7,8,9,max
3206,0.111132,0.197060,0.004090,0.257622,0.100813,0.301478,0.039865,0.064998,0.022301,0.017385,0.301478
934,0.396624,0.337160,0.000665,0.290991,0.034577,0.150206,0.097242,0.010310,0.053773,0.001818,0.396624
2919,0.209947,0.257761,0.001649,0.273799,0.059041,0.212800,0.062262,0.025887,0.034629,0.005621,0.273799
2937,0.209947,0.257761,0.001649,0.273799,0.059041,0.212800,0.062262,0.025887,0.034629,0.005621,0.273799
1501,0.029753,0.099286,0.063459,0.198823,0.316895,0.437846,0.009187,0.249129,0.007806,0.075356,0.437846
...,...,...,...,...,...,...,...,...,...,...,...
2627,0.111132,0.197060,0.004090,0.257622,0.100813,0.301478,0.039865,0.064998,0.022301,0.017385,0.301478
1151,0.205221,0.239321,0.002618,0.255636,0.061304,0.181018,0.046682,0.020185,0.031814,0.003784,0.255636
137,0.111132,0.197060,0.004090,0.257622,0.100813,0.301478,0.039865,0.064998,0.022301,0.017385,0.301478
584,0.111132,0.197060,0.004090,0.257622,0.100813,0.301478,0.039865,0.064998,0.022301,0.017385,0.301478


In [147]:
test["result"]=0
for i in test.index:
    for col in test.columns:
        if test.loc[i,col]==test.loc[i,'max']:
            test.loc[i,'result']=col
            break

In [153]:
pd.merge(test["result"],train_y,right_index=True,left_index=True)

,result,삶의 사다리 점수
3206,5,8
934,0,7
2919,3,7
2937,3,8
1501,5,1
...,...,...
2627,5,5
1151,3,7
137,5,9
584,5,5
